# Using KDB+
* * *
## Load Database
### From Original CSVs

This action only has to be performed once. If you already have the splayed tables saved somewhere, go to section "From Splayed Tables" to load them.

In [1]:
/ Load price tick file
tick:("SIDTFIFIFIS"; enlist"|")0:`:../data/test/raw/test_tick_price_file.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
tickenum: .Q.en[`:../data/test/raw/tickprice/] tick
/ Sample query
select from tickenum where TimeStamp within 09:00:00.00 09:00:10.00
/ Save table
rsave `tickenum

Id         SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskSiz..
-----------------------------------------------------------------------------..
Security_1 1     2022.11.03 09:00:00.000                      10.0312  5200  ..
Security_0 1     2022.11.03 09:00:01.000                                     ..
Security_1 2     2022.11.03 09:00:02.000                                     ..
Security_4 1     2022.11.03 09:00:03.000 22         4800                     ..
Security_0 2     2022.11.03 09:00:04.000 86.0312    1800                     ..
Security_3 1     2022.11.03 09:00:06.000 35         6600                     ..
Security_3 2     2022.11.03 09:00:07.000 35         5400                     ..
Security_4 2     2022.11.03 09:00:08.000 22         8700                     ..
Security_0 3     2022.11.03 09:00:09.000 86.0312    6000                     ..
Security_2 1     2022.11.03 09:00:10.000                      64.0625  2900  ..


`:tickenum/


In [2]:
/ Load base tick file
base:("SSSSS"; enlist"|")0:`:../data/test/raw/test_tick_base_file.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
baseenum: .Q.en[`:../data/test/raw/basetick/] base
/ Sample query
select from baseenum
/ Save table
rsave `baseenum

Id         Ex Descr                          SIC             Cu 
----------------------------------------------------------------
Security_0 LN 'Financial security number: 0' PHARMACEUTICALS JPY
Security_1 LN 'Financial security number: 1' COMPUTERS       FFR
Security_2 LN 'Financial security number: 2' INDUSTRIAL      USD
Security_3 AM 'Financial security number: 3' ENTERTAINMENT   FFR
Security_4 NY 'Financial security number: 4' CHEMICALS       USD


`:baseenum/


### From Splayed Tables

In [3]:
/ Load the tables from saved splayed table using get to save into variable.
baseenum2:get `baseenum / This should be the path to the foldeer with data.
priceenum2:get `tickenum

In [37]:
/ Pending: Figure out how to partition tables and save/load those

## Benchmark Queries

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval

3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

4. Determine the top 10 most active stocks for a specified date sorted by cumulative trade volume by considering all trades

5. Find the most active stocks in the "COMPUTER" industry (use SIC code)

6. Find the 10 stocks with the highest percentage spreads. Spread is the difference between the last ask-price and the last bid-price. Percentage spread is calculated as a percentage of the mid-point price (average of ask and bid price).

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

In [4]:
securities:(`Security_0;`Security_1;`Security_2)
select from priceenum2 where Id in securities, TradeDate=2022.11.03, TimeStamp within 09:00:00.00 09:00:10.00

Id         SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskSiz..
-----------------------------------------------------------------------------..
Security_0 1     2022.11.03 09:00:00.000                      10.0312  5200  ..
Security_1 1     2022.11.03 09:00:01.000                                     ..
Security_0 2     2022.11.03 09:00:02.000                                     ..
Security_2 1     2022.11.03 09:00:03.000 22         4800                     ..
Security_1 2     2022.11.03 09:00:04.000 86.0312    1800                     ..
Security_2 2     2022.11.03 09:00:08.000 22         8700                     ..
Security_1 3     2022.11.03 09:00:09.000 86.0312    6000                     ..


2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval.

3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

In [45]:
a: select TimeStamp:last TimeStamp where not null TradePrice, TradePrice: last TradePrice where not null TradePrice by Id from priceenum2 where TradeDate=2022.11.03
b: select TimeStamp_final: last TimeStamp where not null TradePrice, TradePrice_final:last TradePrice where not null TradePrice by Id  from priceenum2 where TradeDate=2022.11.04
c: a^b
select [10;>(TradePrice_final-TradePrice)%TradePrice] Id, loss:(TradePrice_final-TradePrice)%TradePrice from c

Id         loss      
---------------------
Security_0 0.2280993 
Security_2 0.2045474 
Security_3 0.1681548 
Security_4 0.1302625 
Security_1 0.05972275


4. Determine the top 10 most active stocks for a specified date sorted by cumulative trade volume by considering all trades


In [46]:
select [10;>TradeCumulative] TradeCumulative:sum TradeSize by Id from priceenum2 where TradeDate=2022.11.03 

Id        | TradeCumulative
----------| ---------------
Security_3| 14429100       
Security_4| 14127700       
Security_0| 14018800       
Security_1| 13939600       
Security_2| 13587800       


5. Find the most active stocks in the "COMPUTER" industry (use SIC code)

In [48]:
complete: ej[`Id;baseenum2;priceenum2]
select [1;>TradeCumulative] TradeCumulative:count 1 by Id from complete where SIC like "COMPUTERS"

Id        | TradeCumulative
----------| ---------------
Security_1| 1              


6. Find the 10 stocks with the highest percentage spreads. Spread is the difference between the last ask-price and the last bid-price. Percentage spread is calculated as a percentage of the mid-point price (average of ask and bid price).

In [55]:
a: select  last AskPrice where not null AskPrice, last BidPrice where not null BidPrice by Id from priceenum2 where TradeDate=2022.11.03
select [10;>2*(AskPrice-BidPrice)%(AskPrice+BidPrice)] Id, Percentage_spread: 2*(AskPrice-BidPrice)%(AskPrice+BidPrice) from a

Id         Percentage_spread
----------------------------
Security_2 0.003583997      
Security_4 0.001754386      
Security_0 0                
Security_3 0                
Security_1 -0.0003549868    
